## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Min Temp,Huidity,Cloudiness,Wind Speed,Current Description
0,Flinders,Au,-34.5833,150.8552,59.38,55.26,93,100,7.00,light rain
1,Punta arenas,Cl,-53.1500,-70.9167,32.11,32.11,74,75,9.22,broken clouds
2,Bethel,Us,41.3712,-73.4140,84.88,77.83,42,0,8.05,clear sky
3,Tecpan,Gt,14.7623,-90.9947,54.37,54.37,97,100,5.21,overcast clouds
4,Esperance,Au,-33.8667,121.9000,48.99,48.99,73,0,13.31,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = input("What is the minimum temperature?\n")
max_temp = input("What is the maximum temperature?\n")

What is the minimum temperature?
70
What is the maximum temperature?
95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vc_spots = city_data_df.loc[city_data_df["Max Temp"] < float(max_temp)].loc[city_data_df["Min Temp"] > float(min_temp)]
vc_spots = vc_spots.reset_index(drop=True)
vc_spots.head(10)



,City,Country,Lat,Lng,Max Temp,Min Temp,Huidity,Cloudiness,Wind Speed,Current Description
0,Bethel,Us,41.3712,-73.4140,84.88,77.83,42,0,8.05,clear sky
1,Rikitea,Pf,-23.1203,-134.9692,74.91,74.91,74,11,17.29,few clouds
2,Kavaratti,In,10.5669,72.6420,82.89,82.89,74,100,18.16,overcast clouds
3,Belmonte,Br,-15.8631,-38.8828,78.15,78.15,55,56,9.01,broken clouds
4,Boundiali,Ci,9.5217,-6.4869,77.45,77.45,81,100,5.10,light rain
5,Tamandare,Br,-8.7597,-35.1047,78.76,78.76,77,66,13.42,broken clouds
6,Vaini,To,-21.2000,-175.2000,71.76,71.76,78,40,6.91,scattered clouds
7,Marzuq,Ye,14.4000,46.4667,85.51,85.51,31,28,5.91,scattered clouds
8,Broome,Us,42.2506,-75.8330,79.47,72.37,40,0,12.66,clear sky
9,Porto santo,Pt,33.0720,-16.3415,71.98,71.98,68,20,13.80,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691 entries, 0 to 690
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 691 non-null    object 
 1   Country              691 non-null    object 
 2   Lat                  691 non-null    float64
 3   Lng                  691 non-null    float64
 4   Max Temp             691 non-null    float64
 5   Min Temp             691 non-null    float64
 6   Huidity              691 non-null    int64  
 7   Cloudiness           691 non-null    int64  
 8   Wind Speed           691 non-null    float64
 9   Current Description  691 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 54.1+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No empty rows

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Flinders,Au,59.38,light rain,-34.5833,150.8552,
1,Punta arenas,Cl,32.11,broken clouds,-53.1500,-70.9167,
2,Bethel,Us,84.88,clear sky,41.3712,-73.4140,
3,Tecpan,Gt,54.37,overcast clouds,14.7623,-90.9947,
4,Esperance,Au,48.99,clear sky,-33.8667,121.9000,
5,Hobart,Au,44.28,scattered clouds,-42.8794,147.3294,
6,Rikitea,Pf,74.91,few clouds,-23.1203,-134.9692,
7,Kavaratti,In,82.89,overcast clouds,10.5669,72.6420,
8,Provideniya,Ru,48.83,overcast clouds,64.3833,-173.3000,
9,Portland,Us,65.89,broken clouds,45.5234,-122.6762,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
hotels = []

i = 0

# 6b. Iterate through the hotel DataFrame.
for city in hotel_df["City"]:
    try:
    # 6c. Get latitude and longitude from DataFrame
        params = {
        "radius": 5000,
        "type": "lodging",
        "key": g_key,
        "location": str(hotel_df["Lat"][i]) + ", " + str(hotel_df["Lng"][i])
        }
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
        hotel = requests.get(base_url, params=params).json()["results"][0]["name"]
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotels.append(hotel)
    
        i = i + 1
    except:
        hotels.append("")
        i = i + 1
        

In [9]:
hotel_df["Hotel Name"] = hotels
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Flinders,Au,59.38,light rain,-34.5833,150.8552,Shellharbour Resort & Conference Centre
1,Punta arenas,Cl,32.11,broken clouds,-53.1500,-70.9167,Hotel Hain
2,Bethel,Us,84.88,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
3,Tecpan,Gt,54.37,overcast clouds,14.7623,-90.9947,Villa San Ricardo
4,Esperance,Au,48.99,clear sky,-33.8667,121.9000,Hospitality Esperance
5,Hobart,Au,44.28,scattered clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
6,Rikitea,Pf,74.91,few clouds,-23.1203,-134.9692,People ThankYou
7,Kavaratti,In,82.89,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
8,Provideniya,Ru,48.83,overcast clouds,64.3833,-173.3000,
9,Portland,Us,65.89,broken clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Flinders,Au,59.38,light rain,-34.5833,150.8552,Shellharbour Resort & Conference Centre
1,Punta arenas,Cl,32.11,broken clouds,-53.1500,-70.9167,Hotel Hain
2,Bethel,Us,84.88,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
3,Tecpan,Gt,54.37,overcast clouds,14.7623,-90.9947,Villa San Ricardo
4,Esperance,Au,48.99,clear sky,-33.8667,121.9000,Hospitality Esperance
5,Hobart,Au,44.28,scattered clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
6,Rikitea,Pf,74.91,few clouds,-23.1203,-134.9692,People ThankYou
7,Kavaratti,In,82.89,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
9,Portland,Us,65.89,broken clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
10,Mahebourg,Mu,70.09,light intensity shower rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))